In [ ]:
!pip install krakenex

In [ ]:
#!/usr/bin/env python3

import krakenex
import pandas as pd

# Initialize the Kraken API
api_key = "/f6XxoaKgSVU5ClS3mYLF92bGBaiFPH+qI/x9dUngtVFqpR8b4UdcuWW"
api_sec = "LR/kLjEaFXc/4yiLz6csOMlTRQJVqAecVEPnmLTu12WrytH7rmJNWfuO4tu3ql6pMGVUYkyRw7ObtOCv1AqBgg=="

kraken = krakenex.API()
kraken.key = api_key
kraken.secret = api_sec

# Modifiable Variables
EMA_SHORT_LENGTH = 21
EMA_LONG_LENGTH = 50
ATR_LENGTH = 14
ATR_EMA_LENGTH = 14  # Separate ATR EMA length
ENTRY_HIGH_WINDOW = 15  # Rolling high window for entry
EXIT_LOW_WINDOW = 10  # Rolling low window for exit
RISK_PERCENTAGE = 0.01  # Risk percentage as a decimal (1% of account balance)

# Function to get OHLC data
def get_ohlc_data(pair):
    response = kraken.query_public('OHLC', {'pair': pair, 'interval': 15})
    ohlc_data = response['result'][pair]
    df = pd.DataFrame(
        ohlc_data,
        columns=['time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count']
    )
    df['close'] = df['close'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['open'] = df['open'].astype(float)
    return df

# Function to get USD pairs with leverage
def get_usd_pairs_with_leverage():
    response = kraken.query_public('AssetPairs')
    pairs = response['result']
    usd_pairs = {}

    for k, v in pairs.items():
        if v.get('quote') == 'ZUSD' and v.get('leverage_buy'):
            usd_pairs[k] = {
                'altname': v['altname'],
                'max_leverage': max(v['leverage_buy'])
            }

    return usd_pairs

# Function to get open margin positions
'''def get_open_margin_positions():
    response = kraken.query_private('OpenPositions')
    positions = response['result']
    return [{
        'pair': pos['pair'],
        'entry_price': float(pos['cost']) / float(pos['vol']),
        'volume': float(pos['vol'])
    } for pos in positions.values()]
'''

# Function to get trade balance
def get_trade_balance():
    response = kraken.query_private('TradeBalance', {'asset': 'ZUSD'})
    available_margin = float(response['result']['mf'])
    account_balance = float(response['result']['eb'])
    return available_margin, account_balance

# Function to place a market order for margin trading
def place_market_order(pair, size, action="sell"):
    try:
        response = kraken.query_private('AddOrder', {
            'pair': pair,
            'type': action,
            'ordertype': 'market',
            'volume': -size,
            'oflags': 'fciq'  # Margin trading flag
        })
        if response.get('error'):
            print(f"Error placing market order for {pair}: {response['error']}")
        else:
            print(f"Market order placed for {pair}. Response: {response}")
    except Exception as e:
        print(f"Exception placing market order for {pair}: {e}")

# Function to place a limit order for margin trading
'''def place_limit_order(pair, size, price, action="buy"):
    try:
        response = kraken.query_private('AddOrder', {
            'pair': pair,
            'type': action,
            'ordertype': 'limit',
            'price': price,
            'volume': size,
            'oflags': 'fciq'  # Margin trading flag
        })
        if response.get('error'):
            print(f"Error placing limit order for {pair}: {response['error']}")
        else:
            print(f"Limit order placed for {pair}. Response: {response}")
    except Exception as e:
        print(f"Exception placing limit order for {pair}: {e}")

# Function to calculate position size
def calculate_position_size(account_balance, entry_price, stop_loss, max_leverage, available_margin):
    stop_loss_distance = abs(entry_price - stop_loss)
    position_size = (account_balance * RISK_PERCENTAGE) / stop_loss_distance
    max_size_by_leverage = (max_leverage * available_margin) / entry_price
    return min(position_size, max_size_by_leverage)
'''
# Function to check and exit open positions
def exit_positions(open_positions):
    for pos in open_positions:
        pair = pos['pair']
        df = get_ohlc_data(pair)
        previous_min_low = df['low'].rolling(window=EXIT_LOW_WINDOW, closed='left').min()
        current_close = df.iloc[-1]['close']

        if current_close < previous_min_low.iloc[-1]:
            print(f"Exiting position for {pair}. Current close: {current_close}, Trigger level: {previous_min_low.iloc[-1]}")
            place_market_order(pair, pos['volume'])

# Function to find new entries
'''def calculate_valid_entries(usd_pairs, open_positions, account_balance, available_margin):
    open_pairs = {pos['pair'] for pos in open_positions}
    valid_entries = []

    for pair, details in usd_pairs.items():
        if pair in open_pairs:
            continue  # Skip pairs with open positions or orders

        try:
            df = get_ohlc_data(pair)

            # Technical indicators
            df['EMA_21'] = df['close'].ewm(span=EMA_SHORT_LENGTH, adjust=False).mean()
            df['EMA_50'] = df['close'].ewm(span=EMA_LONG_LENGTH, adjust=False).mean()
            df['ATR_14'] = df['close'].rolling(window=ATR_LENGTH).mean()
            df['ATR_EMA_14'] = df['ATR_14'].ewm(span=ATR_EMA_LENGTH, adjust=False).mean()

            df['Previous_Max_High'] = df['high'].rolling(window=ENTRY_HIGH_WINDOW, closed='left').max()
            df['Previous_Close_Above_Max'] = df['close'].shift(1) > df['Previous_Max_High']

            # Entry conditions
            entry_condition = (
                df.iloc[-1]['EMA_21'] > df.iloc[-1]['EMA_50'] and
                df.iloc[-1]['ATR_14'] > df.iloc[-1]['ATR_EMA_14'] and
                df.iloc[-1]['close'] > df.iloc[-1]['Previous_Max_High'] and
                not df.iloc[-1]['Previous_Close_Above_Max']
            )

            if entry_condition:
                entry_price = df.iloc[-1]['close']
                stop_loss = df['low'].rolling(window=EXIT_LOW_WINDOW, closed='left').min().iloc[-1]
                position_size = calculate_position_size(account_balance, entry_price, stop_loss, details['max_leverage'], available_margin)

                if position_size > 0:
                    valid_entries.append({
                        'Pair': details['altname'],
                        'Entry Price': entry_price,
                        'Stop Loss': stop_loss,
                        'Position Size': position_size,
                        'Max Leverage': details['max_leverage']
                    })

        except Exception as e:
            print(f"Error processing {pair}: {e}")

    return valid_entries

# Main function
def main():
    usd_pairs = get_usd_pairs_with_leverage()
    open_positions = get_open_margin_positions()
    available_margin, account_balance = get_trade_balance()

    # Exit open positions if needed
    print("Checking for exits...")
    exit_positions(open_positions)

    # Calculate new valid entries
    print("Finding new entry opportunities...")
    valid_entries = calculate_valid_entries(usd_pairs, open_positions, account_balance, available_margin)

    # Place limit orders for new entries
    print("\nNew Valid Entries:")
    for entry in valid_entries:
        print(entry)
        place_limit_order(entry['Pair'], entry['Position Size'], entry['Entry Price'])

if __name__ == "__main__":
    main()'''

'def calculate_valid_entries(usd_pairs, open_positions, account_balance, available_margin):\n    open_pairs = {pos[\'pair\'] for pos in open_positions}\n    valid_entries = []\n\n    for pair, details in usd_pairs.items():\n        if pair in open_pairs:\n            continue  # Skip pairs with open positions or orders\n\n        try:\n            df = get_ohlc_data(pair)\n\n            # Technical indicators\n            df[\'EMA_21\'] = df[\'close\'].ewm(span=EMA_SHORT_LENGTH, adjust=False).mean()\n            df[\'EMA_50\'] = df[\'close\'].ewm(span=EMA_LONG_LENGTH, adjust=False).mean()\n            df[\'ATR_14\'] = df[\'close\'].rolling(window=ATR_LENGTH).mean()\n            df[\'ATR_EMA_14\'] = df[\'ATR_14\'].ewm(span=ATR_EMA_LENGTH, adjust=False).mean()\n\n            df[\'Previous_Max_High\'] = df[\'high\'].rolling(window=ENTRY_HIGH_WINDOW, closed=\'left\').max()\n            df[\'Previous_Close_Above_Max\'] = df[\'close\'].shift(1) > df[\'Previous_Max_High\']\n\n            # En

In [ ]:
def calculate_valid_entries(usd_pairs, open_positions, account_balance, available_margin):
    open_pairs = {pos['pair'] for pos in open_positions.values()}
    valid_entries = []

    for pair, details in usd_pairs.items():
        if pair in open_pairs:
            continue  # Skip pairs with open positions

        try:
            #print(f"Processing {details['altname']}...")
            df = get_complete_ohlc_data(pair)

            if df is None or df.empty:
                print(f"No OHLC data for {pair}. Skipping...")
                continue

            # Technical indicators (EMA, ATR, etc.)
            df['EMA_21'] = df['close'].ewm(span=21, adjust=False).mean()
            df['EMA_50'] = df['close'].ewm(span=50, adjust=False).mean()
            df['TR'] = df['high'] - df['low']
            df['ATR_14'] = df['TR'].rolling(window=14).mean()

            # Entry conditions
            df['EMA_Condition'] = df['EMA_21'] > df['EMA_50']
            df['ATR_Condition'] = df['ATR_14'] > df['ATR_14'].ewm(span=14).mean()
            df['Combined_Result'] = df['EMA_Condition'] & df['ATR_Condition']
            df['Max_15_Highs'] = df['high'].rolling(window=15).max()
            df['Entry'] = (df['Combined_Result'] &
                           (df['close'] > df['Max_15_Highs'].shift(1)))

            # If entry conditions are met, calculate position size
            if df.iloc[-1]['Entry']:
                stop_loss = df['low'].rolling(window=10).min().iloc[-2]  # Previous 10-bar low
                close_price = df.iloc[-1]['close']
                risk_per_trade = 0.01 * account_balance  # 1% risk
                position_size = risk_per_trade / (close_price - stop_loss)

                # Validate margin requirements
                trade_value = position_size * close_price
                margin_required = trade_value / details['max_leverage']
                if margin_required > available_margin:
                    print(f"Skipping {pair}: Insufficient margin for trade.")
                    continue

                valid_entries.append({
                    'Pair': details['altname'],
                    'Entry Price': close_price,
                    'Position Size': position_size,
                    'Max Leverage': details['max_leverage'],
                    'Distance to Last 10-Bar Low (%)': round((close_price - stop_loss) / stop_loss * 100, 2),
                    'Stop Loss': stop_loss
                })

        except Exception as e:
            print(f"Error processing {details['altname']}: {e}")

    return valid_entries

In [ ]:
def calculate_position_size(account_balance, entry_price, stop_loss, max_leverage, available_margin):
    # Distance to stop loss (risk per unit)
    stop_loss_distance = abs(entry_price - stop_loss)

    # Risk-based position size (based on 1% risk)
    position_size_in_usd = (account_balance * RISK_PERCENTAGE) / stop_loss_distance
    position_size_in_units = position_size_in_usd / entry_price

    # Leverage-based maximum position size
    max_position_in_usd = available_margin * max_leverage
    max_position_in_units = max_position_in_usd / entry_price

    # Add buffer for fees (e.g., 1% buffer)
    fee_buffer = 1.01  # Adjust as needed
    final_position_size = min(position_size_in_units, max_position_in_units / fee_buffer)
    return round(final_position_size, 8)  # Kraken often allows up to 8 decimal places

In [ ]:
def get_min_order_size():
    response = kraken.query_public('AssetPairs')
    pairs = response['result']
    min_sizes = {}

    for pair, details in pairs.items():
        min_sizes[pair] = float(details.get('ordermin', 0))  # Fetch minimum order size (default to 0 if not found)

    return min_sizes

In [ ]:
def place_limit_order(pair, size, price, available_margin, leverage):
    try:
        # Calculate trade value and margin required
        trade_value = size * price
        margin_required = trade_value / leverage

        # Check if available margin is sufficient
        if margin_required > available_margin:
            print(f"Reducing position size for {pair} to fit margin constraints.")
            size = (available_margin * leverage) / price  # Adjust position size
            trade_value = size * price
            margin_required = trade_value / leverage

        print(f"Trade Value: {trade_value}, Margin Required: {margin_required}, Available Margin: {available_margin}")

        # Place limit order with margin trading and leverage specified
        payload = {
            'pair': pair,
            'type': 'buy',  # Default action is 'buy'
            'ordertype': 'limit',
            'price': price,
            'volume': size,
            'leverage': leverage,  # Explicitly specify leverage
            'oflags': 'fciq',  # Margin trading flag
            'timeinforce': 'GTC'  # Good 'Til Cancelled
        }

        # Log payload before sending
        print(f"Order Payload: {payload}")

        # Send the order
        response = kraken.query_private('AddOrder', payload)

        # Log the full response
        if response.get('error'):
            print(f"Error placing order for {pair}: {response['error']}")
        else:
            print(f"Order placed successfully for {pair}. Response: {response}")

    except Exception as e:
        print(f"Exception placing limit order for {pair}: {e}")

In [ ]:
import pandas as pd

def get_complete_ohlc_data(pair, interval=15):
    try:
        response = kraken.query_public('OHLC', {'pair': pair, 'interval': interval})

        # Log any errors
        if response.get('error'):
            print(f"Error fetching OHLC data for {pair}: {response['error']}")
            return None

        # Extract OHLC data
        ohlc_data = response['result'][pair]

        # Remove the last (incomplete) candle
        complete_candles = ohlc_data[:-1]  # Exclude the last row

        # Convert to a DataFrame for easier processing
        df = pd.DataFrame(complete_candles, columns=[
            'time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count'
        ])

        # Convert numeric columns to float
        df[['open', 'high', 'low', 'close', 'vwap', 'volume']] = df[
            ['open', 'high', 'low', 'close', 'vwap', 'volume']
        ].astype(float)

        return df

    except Exception as e:
        print(f"Exception fetching OHLC data for {pair}: {e}")
        return None

In [ ]:
def close_margin_position(pair, txid, volume, leverage):
    try:
        # Construct the payload with leverage
        payload = {
            'pair': pair,
            'type': 'sell',  # Assuming this is closing a long position
            'ordertype': 'market',
            'volume': volume,  # Set the exact volume to close
            'position': txid,  # Specify the position ID to close
            'leverage': leverage,  # Add the correct leverage
            'oflags': 'fciq'  # Optional: Prefer fees in quote currency
        }

        # Log the payload
        print(f"Close Order Payload: {payload}")

        # Send the order to Kraken
        response = kraken.query_private('AddOrder', payload)

        # Log the response
        if response.get('error'):
            print(f"Error closing position for {pair}: {response['error']}")
        else:
            print(f"Position closed successfully for {pair}. Response: {response}")

    except Exception as e:
        print(f"Exception closing position for {pair}: {e}")

In [ ]:
def get_open_margin_positions():
    try:
        response = kraken.query_private('OpenPositions')
        if response.get('error'):
            print(f"Error retrieving open positions: {response['error']}")
            return {}

        positions = response['result']

        # Parse positions and calculate leverage
        open_positions = {}
        for txid, details in positions.items():
            pair = details['pair']
            volume = float(details['vol'])  # Volume of the position
            cost = float(details['cost'])  # Total cost of the position
            margin = float(details.get('margin', 0))  # Margin held by Kraken

            # Calculate leverage dynamically
            leverage = round(cost / margin, 0) if margin > 0 else '1'

            # Calculate per-unit entry price (optional)
            entry_price = round(cost / volume, 6) if volume > 0 else 0

            open_positions[txid] = {
                'pair': pair,
                'volume': volume,
                'cost': cost,  # Total cost of the position
                'entry_price': entry_price,  # Calculated per-unit price
                'margin': margin,
                'leverage': int(leverage)  # Add calculated leverage
            }

        return open_positions

    except Exception as e:
        print(f"Exception retrieving open margin positions: {e}")
        return {}

In [ ]:
def main():
    # Step 1: Retrieve open margin positions with calculated leverage
    open_positions = get_open_margin_positions()
    print(open_positions)

    # Step 2: Check for exits on open positions
    print("\nChecking for exits on open positions...")
    for txid, position in open_positions.items():
        pair = position['pair']
        volume = position['volume']
        leverage = position['leverage']  # Dynamically calculated leverage
        ohlc_data = get_complete_ohlc_data(pair, interval=15)

        if ohlc_data is None or ohlc_data.empty:
            print(f"No OHLC data for {pair}. Skipping...")
            continue

        # Example exit condition: close below previous 10-bar lows
        last_close = float(ohlc_data.iloc[-1]['close'])  # Last complete candle close
        ten_bar_low = ohlc_data['low'].rolling(window=10).min().iloc[-2]  # Use previous 10 bars
        if last_close < ten_bar_low:
            print(f"Exit condition met for {pair}. Closing position...")
            close_margin_position(pair, txid, volume, leverage)

    # Step 3: Retrieve trading pairs and account info
    usd_pairs = get_usd_pairs_with_leverage()
    available_margin, account_balance = get_trade_balance()
    min_order_sizes = get_min_order_size()

    # Step 4: Find new entry opportunities
    print("\nFinding new entry opportunities...")
    valid_entries = calculate_valid_entries(
        usd_pairs=usd_pairs,
        open_positions=open_positions,
        account_balance=account_balance,
        available_margin=available_margin
    )

    # Step 5: Display valid entries
    print("\nNew Valid Entries:")
    for entry in valid_entries:
        print(entry)

    # Step 6: Validate and place limit orders for valid entries
    for entry in valid_entries:
        pair = entry['Pair']
        entry_price = entry['Entry Price']
        position_size = entry['Position Size']
        leverage = entry['Max Leverage']

        trade_value = position_size * entry_price
        margin_required = trade_value / leverage

        print(f"\nPlacing limit order for {pair}...")
        print(f"Details: Entry Price: {entry_price}, Position Size: {position_size}, USD Value: {trade_value}")
        print(f"Available Margin: {available_margin}, Margin Required: {margin_required}")

        if margin_required > available_margin:
            print(f"Skipping {pair}: Insufficient margin for the trade.")
            continue

        if position_size < min_order_sizes.get(pair, 0):
            print(f"Skipping {pair}: Position size {position_size} is below minimum of {min_order_sizes.get(pair, 0)}")
            continue

        place_limit_order(pair, position_size, entry_price, available_margin, leverage)

    print("\nExecution Complete. Check logs for details.")

main()

{}

Checking for exits on open positions...

Finding new entry opportunities...

New Valid Entries:

Execution Complete. Check logs for details.
